In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Problem Set 3

## Øvelse 1: Implementer manuelt OLS-estimatoren

### Del 1: OLS i SLR-tilfældet (en enkelt forklarende variabel)


Når vi kun har 1 forklarende variabel og en konstant (SLR-tilfældet) arbejder vi med denne model:

$$ y_i = \beta_0 + \beta_1 x_i + u_i $$

Da ved vi fra den økonometriske teori at den sande værdi af $\beta_1$ er givet ved:

$$ \beta_1 = \frac{{\text{Cov}}(x, y)} {\text{Var}(x)}  $$

Vi ved også fra forelæsningerne at vi kan estimere covariansen og variansen sådan her:

\begin{align*}
\widehat{\text{Cov}}(x)  = \frac{1}{n} \sum (x_i - \bar{x})(y_i - \bar{y}) \\
\widehat{\text{Var}}(x) = \frac{1}{n} \sum (x_i - \bar{x})^2
\end{align*}

hvor $\bar{x}$ og $\bar{y}$ er de empiriske gennemsnit af $x, y$ i vores datasæt.  Sætter vi alt dette sammen ved hjælp af Analogi-princippet betyder det at vi kan estimere $\hat{\beta}_1$ med formlen:
$$
\hat{\beta}_1 = \frac{\sum (x_i - \bar{x})(y_i - \bar{y}) }{ \sum (x_i - \bar{x})^2}
$$

Dette er OLS-estimatoren for $\hat{\beta}_1$ i SLR-tilfældet.



#### Opgave 1 
Skriv en funktion `OLS_SLR()`, hvor du manuelt implementerer OLS-estimatoren for $\hat{\beta}_1$ i SLR-tilfældet. Brug numpy i din implementering. Funktionen skal tage x og y som argumenter, hvor x er et $n \times 1$ numpy array og y er et $n \times 1$ numpy array. Funktionen skal returnere skalar-værdien $\hat{\beta_1}$.

_Hints:_ 
- Du kan bruge numpy funktionen `np.mean()` til at beregne gennemsnittet af et numpy array og `np.sum()` til at beregne summen over et numpy array.



In [2]:
def OLS_SLR(x,y):
    x_bar = np.mean(x)
    y_bar = np.mean(y)

    cov_xy = np.sum((x - x_bar)*(y - y_bar)) 
    var_x  = np.sum((x - x_bar)**2)

    beta_1_hat = cov_xy / var_x

    return beta_1_hat

#### Opgave 2
Her er noget meget simpelt testdata du kan bruge til at tjekke om din implementering er korrekt. Når du kører din funktion bør du få estimatet $\hat\beta_1 = 2.0$


In [3]:
X = np.array([ 1, 2, 3, 4, 5,])
Y = np.array([-4, 1, 3, 5, 4,])

OLS_SLR(X,Y)

2.0

#### Opgave 3 
Fra den økonometriske teori ved vi også, at vi kan estimere $\beta_0$ med formlen:
$$ \hat{\beta}_0 = \bar{y} - \hat{\beta}_1 \bar{x} $$

Reflekter et øjeblik over formlen: Det giver intuitivt ret god mening. Estimatet af interceptet er gennemsnittet af vores $y$ observationer minus den estimerede effekt af den gennemsnitlige x-observation. 

Tegn et tilfældigt scatter plot på et stykke papir, tegn den bedste lige linje (en OLS-linje) igennem observationerne og se selv hvor den skærer y-aksen.

#### Opgave 4
Udvid din `OLS_SLR()` funktion så den returnerer ikke bare $\hat{\beta}_1$, men en tuple der indeholder ${(\hat\beta_0, \hat\beta_1)}$. Når du kører din udvidede funktion på samme testdata som før bør du få estimaterne ${(\hat\beta_0 = -4.2, \hat\beta_1 = 2.0)}$

In [4]:
def OLS_SLR(x,y):
    x_bar = np.mean(x)
    y_bar = np.mean(y)
    n = len(x)

    cov_xy = np.sum((x - x_bar)*(y - y_bar)) / n
    var_x  = np.sum((x - x_bar)**2) / n

    beta_1_hat = cov_xy / var_x
    beta_0_hat = y_bar - beta_1_hat*x_bar

    return (beta_0_hat.round(4), beta_1_hat.round(4))

OLS_SLR(X,Y)

(-4.2, 2.0)

Det sidste trin er at beregne standardfejlene for vores OLS-estimater.  For at gøre dette skal vi først beregne residualerne med formlen:
$$ \hat{u_i} = y - \beta_0 - \beta_1 x $$

Vi bruger residualerne til at estimere residualvariansen:

$$
\hat{\sigma}^2 = \frac{1}{n-2} \sum_{i=1}^{n} \hat{u}_i^2
$$

som vi kan bruge til at beregne variansen af OLS-estimatorerne:
$$
\text{Var}(\hat{\beta}_1 \mid X) = \frac{\hat{\sigma}^2}{\sum_{i=1}^{n} (x_i - \bar{x})^2}
$$
$$
\text{Var}(\hat{\beta}_0 \mid X) = \frac{\hat{\sigma}^2 \cdot \frac{1}{n}\sum_{i=1}^{n}x_{i}^{2}}{\sum_{i=1}^{n}(x_i - \bar{x})^2}
$$

Standardfejlene er ganske enkelt kvadratrødderne af varianserne:
$$
\text{se}(\hat{\beta}_1) = \sqrt{\text{Var}(\hat{\beta}_1 \mid X)}
$$
$$
\text{se}(\hat{\beta}_0) = \sqrt{\text{Var}(\hat{\beta}_0 \mid X)}
$$




#### Opgave 5
Udvid din `OLS_SLR()` funktion så den også returnerer standardfejlene af koefficienterne. Når du kører funktionen på samme testdata som før skulle du gerne få:

$$ \hat \beta_0 = -4.2, \quad \hat \beta_1 = 2.0, \quad \text{se}(\hat \beta_0) = 1.99, \quad \text{se}(\hat \beta_1) = 0.6  $$

_Hint_: Brug funktionen `np.sqrt()` til at tage kvadratroden af et tal.

In [5]:
def OLS_SLR(x,y):
    x_bar = np.mean(x)
    y_bar = np.mean(y)
    n = len(x)

    cov_xy = np.sum((x - x_bar)*(y - y_bar)) / n
    var_x  = np.sum((x - x_bar)**2) / n

    beta_1_hat = cov_xy / var_x
    beta_0_hat = y_bar - beta_1_hat*x_bar

    # Compute residual variance
    residuals = y - (beta_0_hat + beta_1_hat * x)
    sigma_squared_hat = np.sum(residuals**2) / (n-2)

    # Compute variance and standard errors for the OLS-estimators
    SST_x = np.sum((x - x_bar)**2)
    var_beta1 = sigma_squared_hat / SST_x
    var_beta0 = sigma_squared_hat * np.mean(x**2) / SST_x

    se_beta1 = np.sqrt(var_beta1)
    se_beta0 = np.sqrt(var_beta0)

    return (beta_0_hat.round(4), 
            beta_1_hat.round(4),
            se_beta0.round(4),
            se_beta1.round(4))

OLS_SLR(X,Y)

(-4.2, 2.0, 1.99, 0.6)

#### Opgave 6 
Brug `sm.OLS()` funktionen fra statsmodels-pakken til at estimere samme model på testdataen og bekræft at dine koefficienter og standardfejl er de samme som dem statsmodels returnerer.

In [6]:
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     11.11
Date:                Fri, 13 Sep 2024   Prob (F-statistic):             0.0446
Time:                        09:37:26   Log-Likelihood:                -9.0200
No. Observations:                   5   AIC:                             22.04
Df Residuals:                       3   BIC:                             21.26
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2000      1.990     -2.111      0.1

/Users/danielriegels/anaconda3/lib/python3.11/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "



### Del 2: OLS i MLR-tilfældet (Når vi har flere forklarende variable)


I det generelle tilfælde hvor vi har flere forklarende variable (MLR), ser modellen sådan her ud:

$$y_i = \beta_0 + \beta_1 x_{1 i}+ \beta_2 x_{2 i} +  \dots + \beta_k x_{k i} + u_i$$

Da ved vi at OLS-estimatoren kan skrives på matrix-form på følgende måde:

$$ \boldsymbol{\hat {\beta}} = (\mathbf {X'} \mathbf X)^{-1} \mathbf {X'} \mathbf y $$

hvor:
- $\boldsymbol{\hat{\beta}}$ er en $ (k+1) \times 1 $ vektor af koefficient estimater.
- $\mathbf X$ er en $ n \times (k+1) $ matrix indeholdende vores observationer af forklarende variable, hvor den første søjle er fyldt med 1-taller for at fange konstante effekter.
- $\mathbf y$ er en $ n \times 1 $ vektor som indeholder vores observationer af de afhængige variable

$n$ er antallet af observationer. $k$ er antallet af forklarende variable (vi plusser 1 for at gøre plads til konstantleddet).



#### Opgave 1
Skriv en funktion `OLS()`, hvor du manuelt implementerer OLS estimatoren i MLR-tilfældet ved hjælp af numpy. Funktionen skal tage X og Y som argumenter, hvor X er et $n \times (k+1)$ numpy array og Y er et $n \times 1$ numpy array. Funktionen skal returnere et $(k+1) \times 1$ numpy array $\hat{\beta}$.

_Hints:_ 
- Du kan gange matricer sammen i numpy ved hjælp af @ operatoren. Hvis du for eksempel har to kompatible matricer A og B, kan du bruge koden `A @ B` til at beregne matrix produktet. @ operatoren fungerer også til matrix-vector produkter.

- Du kan transponere en matrix i numpy ved at tilføje `.T` til sidst. For eksempel returnerer `A.T` den transpose af matricen A.
- Du kan invertere en matrix med `np.linalg.inv()` metoden. For eksempel returnerer `np.linalg.inv(A)` den inverse af matricen A.

**Din kode:** 

In [7]:
def OLS(X,Y):
    beta_hat = np.linalg.inv(X.T @ X) @ X.T @ Y
    return beta_hat

#### Opgave 2 
Tjek at `OLS()` funktionen returnerer de samme koefficienter som i SLR-tilfældet når du kører den på samme data. Kør bare cellen nedenfor.

In [8]:
OLS(X,Y)

array([-4.2,  2. ])

#### Opgave 3 
Lad os nu tjekke om funktionen også virker når vi har flere forklarende variable (flere søjler i X). Hvis din implementering er rigtig bør du få  følgende koefficient vektor $\hat{\beta}$: `array([-3.58043168,  1.61631032,  0.30264712])` når du kører nedenstående celle.

In [9]:
X = np.array([[ 1, 2, 3, 4, 5, 1, 0, 1],
              [-2, 5, 2, 1, 5, 7, 5, 2]]).T
X = sm.add_constant(X)
Y = np.array([-4, 1, 3, 5, 4, 1, -2.6, -1])

OLS(X,Y)

array([-3.58043168,  1.61631032,  0.30264712])

## Øvelse 2: SLR, MLR og makrodata
I denne opgave skal vi øve os i at formulere nul- og alternativhypoteser. Samtidig skal vi sammenligne en simpel regressionsmodel (SLR) med en multipel regressionsmodel (MLR).

Specifikt vil vi kigge på de makroøkonomiske hypoteser om om _absolut_ og _betinget_ konvergens. Teorierne omhandler langsigtet økonomisk vækst, og om lande kan vokse ud af fattigdom uden indgreb. 

- Teorien om **absolut konvergens** siger, at på lang sigt vil BNP pr. arbejder konvergere mod samme vækstbane. Det betyder, at jo længere væk et lands initiale BNP pr. arbejder er fra den langsigtede vækstbane, desto hurtigere vil BNP pr. arbejder vokse for at nå samme niveau. 

- Teorien om **betinget konvergens** siger, at et lands BNP pr. arbejder konvergerer mod en lande-specifik vækstbane, som afhænger af landets strukturelle karakteristika (fx befolkningstilvæksten og inveresteringsraten i fysisk kapital, som kan variere fra land til land)

Vi bruger datasættet PS3.dta, som indholder vækstdata for 93 lande fra 1960-2003. Dataen stammer fra Penn World Tables.

### Del 1: Simpel regressionsmodel (SLR)
I første omgang vil vi arbejde med følgende simple regressionsmodel:

$$ \text{gy}_i = \beta_0 + \beta_1 \ln(\text{y60}_i) + u_i $$

hvor:
- $\text{gy}_i $ er den gennemsnitlige årlige vækstrate i BNP pr. arbejder for land $i$ siden 1960
- $\text{y60}_i$ er landets BNP pr. arbejder i år 1960.

Bemærk, at vi ligesom i sidste uge tager log-transformationen af BNP pr. arbejder i 1960 for at opnå bedre numeriske resultater, og for at få en procentuel fortolkning af koefficient-estimatet.

#### Opgave 1
Hvad er nul- og alternativ hypotesen, når I gerne vil teste hypotesen om absolut konvergens? Vær præcis, når I formulerer nul- og alternativ hypotesen i ord.

> Nulhypotesen er, at der ikke er absolut konvergens, så $H_0: \beta_1 = 0$. Det vil sige, at landets vækstrate ikke afhænger af landets oprindelige BNP-per-arbejder-niveau i 1960.
> 
> Alternativ-hypotesen er, at der _er_ absolut konvergens, så $H_A : \beta_1 < 0$. At der er absolut konvergens vil betyde, at alle lande konvergerer mod samme BNP pr. arbejder. Det betyder, at lande, der var rige i 1960, skal have en lavere vækstrate i BNP pr. arbejder end lande, som var fattige i 1960, sådan at de fattige lande "automatisk" vil indhente de rige over tid.
>
> Bonusinfo til de nysgerrige:
> 1. Algebraisk kan vi omskrive $\beta_1 = -b$ så ligningen bliver:
>$$\text{gy}_i = \beta_0 - b \ln(\text{y60}_i) + u_i$$
>2. Leddet $-b \ln(\text{y60}_i)$ kan omskrives med en logaritmeregneregel:
>$$-b \ln(\text{y60}_i) = \ln((\text{y60}_i)^{-b}) = \ln\left(\frac{1}{\text{y60}_i^b}\right)$$
>3. Det viser at hvis $\text{y60}_i$ er høj for et specifikt land (dvs. et land der starter med et higher initialniveau af BNP), da bliver det logaritmiske led lavere (siden $\frac{1}{\text{y60}_i^b}$ falder), hvilket fører til en lavere vækstrate $\text{gy}_i$. Det fører til konvergens.


#### Opgave 2
Indlæs datasættet PS3.dta og drop de observationer som har datakvalitet D i kolonnen "grade". Hvilke lande bliver fravalgt? Hvilken betydning for analysen om absolut konvergens kan det have, at vi frasorterer lande med lav datakvalitet?

In [10]:
df = pd.read_stata("PS3.dta")

# Lande med grade D
df[df.grade == 'D']

,country,iso3,y03,y60,gy,sk,sh,u,isi,n,grade
23,Taiwan,TWN,0.627780,0.122368,0.055736,0.170054,NaN,8.764,0.76689,0.022051,D
40,Algeria,DZA,0.239503,0.394755,0.006088,0.156739,0.124225,NaN,0.26467,0.030309,D
48,Cape Verde,CPV,0.179736,0.134228,0.024498,0.148722,NaN,NaN,0.22907,0.022813,D
72,Lesotho,LSO,0.069507,0.039794,0.030679,0.174102,NaN,4.232,0.55154,0.016500,D
79,Mozambique,MOZ,0.040890,0.045656,0.015145,0.029996,0.009497,1.105,0.26800,0.018978,D
80,Comoros,COM,0.039927,0.089702,-0.001115,0.113735,NaN,NaN,0.50838,0.029056,D
84,Uganda,UGA,0.033853,0.050269,0.008515,0.026438,0.014670,3.514,0.26178,0.026821,D
87,Chad,TCD,0.028558,0.070628,-0.003349,0.085335,0.008054,NaN,0.27702,0.023534,D
88,Togo,TGO,0.027333,0.058020,0.000204,0.099361,0.033496,3.334,0.22283,0.029585,D
89,Niger,NER,0.026838,0.072005,-0.005243,0.059065,0.007126,1.018,0.25700,0.026862,D


In [11]:
# Deskriptiv statistik for hele datasættet
df.describe()

,y03,y60,gy,sk,sh,u,isi,n
count,97.000000,97.000000,97.000000,97.000000,83.000000,81.000000,96.000000,97.000000
mean,0.326837,0.306645,0.016238,0.157079,0.066243,6.478321,10.909232,0.021910
std,0.331110,0.277975,0.013516,0.081959,0.036105,2.831886,102.011581,0.008936
min,0.018935,0.026356,-0.010774,0.024205,0.005237,0.839000,0.193390,0.000795
25%,0.069507,0.072005,0.006755,0.088349,0.033082,4.796000,0.275627,0.016397
50%,0.179736,0.223580,0.014864,0.137713,0.073028,6.355000,0.396475,0.023534
75%,0.586415,0.464846,0.024969,0.224658,0.095137,8.732000,0.791185,0.028721
max,1.735000,1.030000,0.055736,0.435997,0.146492,12.049000,1000.000000,0.047924


In [12]:
# Deskriptiv statistik for grade == D lande
df[df.grade == 'D'].describe()

,y03,y60,gy,sk,sh,u,isi,n
count,11.000000,11.000000,11.000000,11.000000,6.000000,7.000000,11.000000,11.000000
mean,0.121169,0.100705,0.012538,0.108679,0.032845,3.258000,0.351445,0.023839
std,0.182800,0.102940,0.018285,0.053455,0.045831,2.794059,0.177304,0.005175
min,0.018935,0.030327,-0.005243,0.026438,0.007126,0.839000,0.222830,0.015722
25%,0.027945,0.047962,-0.000455,0.072200,0.008415,1.061500,0.257860,0.020515
50%,0.039927,0.070628,0.006755,0.113735,0.012083,3.334000,0.264670,0.023534
75%,0.124621,0.106035,0.019822,0.152730,0.028789,3.873000,0.392700,0.027959
max,0.627780,0.394755,0.055736,0.174102,0.124225,8.764000,0.766890,0.030309


In [13]:
df = df[df.grade != "D"] # Fjern lande med grade D

**Dit svar:**


> Det er lande med lavt BNP pr. arbejder og lande, der har haft lavere vækst i perioden ift. resten af datasættet, som bliver fravalgt. Det kan have en betydning for analysen af absolut konvergens. Det vil ikke have en betydning, hvis de frasorterede lande lignende landene i resten af datasættet.

#### Opgave 3
Konstruer variablen $\ln (\text{y60}_i)$ og estimer modellen med `sm.OLS()`.  Hvad er modellens forklaringsgrad, $R^2$? Hvad kan I konkludere om absolut konvergens?

**Din kode:**

In [14]:
Y = df.gy
X = np.log(df.y60)

X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     gy   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.1867
Date:                Fri, 13 Sep 2024   Prob (F-statistic):              0.667
Time:                        09:37:26   Log-Likelihood:                 253.08
No. Observations:                  86   AIC:                            -502.2
Df Residuals:                      84   BIC:                            -497.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0158      0.002      6.378      0.0

**Dit svar:**

> Modellens forklaringsgrad er 0,2 pct. Det er ikke muligt at konkludere, at der er en sammenhæng mellem BNP pr. arbejder i 1960 og vækst i BNP pr. arbejder. Hypotesen om absolut konvergens bliver ikke understøttet empirisk.

### Del 2: Multipel regressionsmodel
Vi udvider nu modellen til at inkludere en variabel, som tager højde for forskelle i strukturelle karakteristika på tværs af lande:

$$ \text{gy}_i = \beta_0 + \beta_1 \ln(\text{y60}_i) + \beta_2 \text{struc}_i + u_i $$
hvor de strukturelle karakteristika er defineret på følgende måde:
$$ \text{struc}_i = \ln(\text{sk}_i) − \ln(\text{n}_i + 0, 075)$$ 

Da I endnu ikke har haft Makroøkonomi A behøver I ikke forstå modellen fuldt ud, men det kan nævnes at:
- $\text{sk}_i$ er investeringsraten i fysisk kapital for land $i$

- $\text{n}_i$ er den gennemsnitlige årlige befolkningsvækst i perioden 1960-2003
- De 0,075 i det sidste led er et estimat for afskrivningsraten af fysisk kapital i den basale Solow model. Bemærk, at det er samme værdi for alle lande. I lærer mere om alt dette i Makro A.

#### Opgave 4
Hvad er nul- og alternativhypotesen, når vi gerne vil teste hypotesen om _betinget_ konvergens?

**Dit svar:** 

> Nulhypotesen er, at der ikke er betinget konvergens: $H_0: \beta_2 = 0$. Det betyder, at vækstraten ikke afhænger af de strukturelle karakteristika. Alternativhypotesen er $H_A: \beta_2 \neq 0$. Det betyder, at strukturelle karakteristika har betydning for vækstraten.

#### Opgave 5
Konstruer den nye variabel $\text{struc}_i$. Estimer den multiple regressionsmodel. Hvor meget af variationen i den afhængige variable er forklaret af modellen? Fortolk koefficienten for det initiale niveau af BNP pr. capita. Hvad kan vi konkludere om betinget konvergens? Overvej om en anden afskrivningsrate vil have betydning for jeres estimater.

**Din kode:**

In [15]:
df['struc'] = np.log(df.sk) - np.log(df.n + 0.075)
df['ln_y60'] = np.log(df.y60)

Y = df.gy
X = df[['ln_y60', 'struc']]
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     gy   R-squared:                       0.428
Model:                            OLS   Adj. R-squared:                  0.415
Method:                 Least Squares   F-statistic:                     31.10
Date:                Fri, 13 Sep 2024   Prob (F-statistic):           8.32e-11
Time:                        09:37:26   Log-Likelihood:                 277.04
No. Observations:                  86   AIC:                            -548.1
Df Residuals:                      83   BIC:                            -540.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0022      0.003      0.854      0.3

**Dit svar:**

> Forklaringsgraden er 42,8 pct. Fortolkningen af $\hat\beta_1$ er, at hvis BNP pr. arbejder er 1 pct. større i 1960, vil den  årlige vækstrate i BNP pr. arbejder fra 1960 til 2003 være -0,0055 pct. point lavere. Hvis nulhypotesen kan forkastes, er der empirisk evidens for hypotesen om betinget konvergens, og lande med samme strukturelle karakteristika vil konvergere mod samme BNP pr. arbejder.
> 
> [Bemærk, at modellen skal fortolkes som en level-log model, men at man i fortolkningen ikke dividerer med 100, så man fortolker ændringen i pct. point].


#### Opgave 6
I denne opgave vil du lære hvordan du let kan eksportere output fra statsmodels og fra pandas DataFrames til LaTeX, sådan at du kan bruge output fra Python i pæne tabeller, der er korrekt sat op i dine afleveringsopgaver.


- Brug denne kommando i Python til at printe dit regression summary som LaTeX kode:
```py
print(results.summary().as_latex())
```

- Opret et LaTeX-dokument (fx i Overleaf).

- Kopier koden ind i dit LaTeX dokument og se, hvordan tabellen ser ud.

Tilsvarende kan du konvertere ethvert Pandas DataFrame (fx deskriptiv statistik) til LaTeX med metoden `.to_latex()`. Bemærk at navngivningen af de to metoder er lidt forskellig i statsmodels og i pandas.

Prøv fx at køre koden:
```py
print(df.describe().to_latex())
```

*Tip:* Hvis du vil afrunde dit LaTeX output til fx to decimaler, kan du bruge argumentet `float_format="%.2f"` i `.to_latex()` metoden

*Tip:* Din tabel bliver automatisk flottere i LaTeX hvis du importerer LaTeX pakken booktabs:
```\usepackage{booktabs}```

**Din kode:**

In [16]:
print(df.describe().to_latex(float_format="%.2f"))

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & y03 & y60 & gy & sk & sh & u & isi & n & struc & ln_y60 \\
\midrule
count & 86.00 & 86.00 & 86.00 & 86.00 & 77.00 & 74.00 & 85.00 & 86.00 & 86.00 & 86.00 \\
mean & 0.35 & 0.33 & 0.02 & 0.16 & 0.07 & 6.78 & 12.28 & 0.02 & 0.38 & -1.56 \\
std & 0.34 & 0.28 & 0.01 & 0.08 & 0.03 & 2.66 & 108.41 & 0.01 & 0.63 & 1.06 \\
min & 0.02 & 0.03 & -0.01 & 0.02 & 0.01 & 0.88 & 0.19 & 0.00 & -1.38 & -3.64 \\
25% & 0.09 & 0.08 & 0.01 & 0.09 & 0.04 & 5.08 & 0.31 & 0.02 & -0.04 & -2.53 \\
50% & 0.19 & 0.26 & 0.02 & 0.15 & 0.07 & 6.57 & 0.41 & 0.02 & 0.44 & -1.36 \\
75% & 0.66 & 0.52 & 0.02 & 0.23 & 0.10 & 8.80 & 0.81 & 0.03 & 0.93 & -0.65 \\
max & 1.74 & 1.03 & 0.05 & 0.44 & 0.15 & 12.05 & 1000.00 & 0.05 & 1.39 & 0.03 \\
\bottomrule
\end{tabular}



Du vil måske opleve, at det kræver lidt mere arbejde at få tabellen til at se godt ud i LaTeX. Blandt andet fordi statsmodels faktisk spytter flere tabeller af forskellig størrelse ud, når man printer et summary.

Du kan bede statsmodels om kun at printe tabellen med koefficient-estimaterne sådan her:

```py
print(results.summary().tables[1])
```

**Sidste tip til de "dovne":** Hvis du har travlt og ikke har lyst til at rode med at sætte LaTeX-tabellerne pænt op, kan du også bare kopiere de rå tekstoutput fra Python og indsætte i et verbatim miljø i LaTeX – det ser OK ud. Du skriver følgende i LaTeX:

```LaTeX
\begin{verbatim}
<Dit rå textoutput her>
\end{verbatim}
```